In [ ]:
import math
from multiprocessing import Process, Queue
from Queue import Empty, Full
import sys
import time
import traceback
import msgpack
import logbook
import network.bfs

In [ ]:
exp_name = "20_find_path_length"
edges_file = "archive/17_create_coeditor/2017-04-26 14:53:15 1b4e2e0/%d-coeditor.mp"
project_ids = [861,874,1846,2234,1813,2042,1285,529,116,66,2149,1453,1306,1072,1666,1626,898,2224,762,1339,2265,2059,1076,1070,1767,844,136,2047,949,1668,1366,1958,2078,2034,1425,1991,392,1044,1903,365,1073,1337,1336,606,1042,2092,163,317,371,1553,2259,1492,2084,2215,1334,1763,366,2210,998,1520,1943,570,282,1774,1080,1137,147,899,360,307,1544,684,2141,834,655,53,63,947,987,1853,779,828,1777,1804,240,416,2159,452,916,903,652,1191,1682,2104,2211,629,824,2269,1936,1496,974,1632,475,2191,2076,1616,2095,196,1750,619,1344,393,137,2144,869,795,2030,518,748,2140,1483,2228,536,1233,882,102,1941,274,1603,23,1990,585,1963,280,2222,1061,537,248,2125,1421,447,1790,1247,1513,227,556,1576,1451,28,1658,1579,626,1981,2196,1766,580,1014,36,991,1769,728,198,963,1105,1857,1746,1350,2155,382,1654,946,817,387,1989,715,2082,1741,161,1162,1675,441,1449,2088,42,770,1948,1756,2235,1038,1974,38,1194,1378,1572,1177,2131,1417,543,1095,880,876,1045,540,2139,2122,1681,1447,1327,1444,551,1693,716,1645,1890,341,2156,1887,1370,1640,515,417,1720,1463,985,653,2172,486,209,113,1629,941,523,2077,1946,589,1515,190,1411,1160,2231,1468,1768,1149,1122,40,855,1805,1004,1547,1490,887,982,1397,601,645,522,1843,2237,1361,1892,1508,1954,2127,1697,1456,1855,1860,1620,1475,1294,1278,1923,457,1980,273,893,1761,337,329,915,1728,37,245,2255,131,814,1090,489,204,1335,219,1811,418,1097,1375,191,2177,2012,1998,798,2123,499,1157,144,347,1303,1994,1057,179,2207,1464,128,600,646,1984,1276,1470,1930,443,774,1724,2014,126,2175,2226,1260,1493,1526,1261,1931,881,1067,803,690,1504,1744,1280,1584,502,405,591,1867,1912,1791,356,2060]

In [ ]:
exp = logbook.Experiment(exp_name)
log = exp.get_logger()
try:
    with open(exp.get_filename("reachability")) as f_reach:
        f_reach.write("project_id,frac_reach,min_reach,max_reach,node_count\n")
        for project_id in project_ids:
            all_nodes = set()
            edge_count = 0
            project_id = 34
            edges_from = {}
            reachable = 0
            min_reachable = 0
            max_reachable = 0
            log.info("Loading network edges for project %d " % (project_id,))
            with open(edges_file % project_id, "rb") as f:
                unpacker = msgpack.Unpacker(f)
                for o in unpacker:
                    edge_count += len(o[1])
                    edges_from[o[0][0]] = set(o[1])
                    all_nodes.add(o[0][0])
                    all_nodes |= set(o[1])
            all_nodes = list(all_nodes)
            node_count = len(all_nodes)
            log.info("  Loaded %d nodes and %d edges" % (node_count, edge_count))
            total_time = 0.0
            for i, source in enumerate(all_nodes):
                if i > 0:
                    est = total_time * len(all_nodes) / float(i)
                    log.info("%d complete in %f seconds, total estimated: %f" % (i, total_time, est))
                start = time.time()
                distances = network.bfs.get_distances_bfs(edges_from, all_nodes[i])
                total_time += time.time() - start
                source_reachable = len(distances)
                reachable += source_reachable
                if min_reachable > source_reachable or min_reachable < 0:
                    min_reachable = source_reachable
                if max_reachable < source_reachable or max_reachable < 0:
                    max_reachable = source_reachable
            p_reachable = reachable / float(node_count*(node_count-1))
            f_reach.write("%d,%f,%f,%f,%d" % (
                project_id, p_reachable,
                min_reachable / float(node_count),
                max_reachable / float(node_count),
                node_count
            ))
            f_reach.flush()
            log.info("  Project %d complete" % project_id)
except KeyboardInterrupt:
    log.info("Keyboard interrupt")
    raise